In [2]:
import random
import csv

# ------------------ Activation Functions ------------------ #
def sigmoid(x):
    return 1 / (1 + exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

def exp(x, terms=10):
    result = 1.0
    numerator = 1.0
    denominator = 1.0
    for i in range(1, terms):
        numerator *= x
        denominator *= i
        result += numerator / denominator
    return result

# ------------------ Layer Class ------------------ #
class Layer:
    def __init__(self, input_size, output_size):
        self.weights = [[random.uniform(-1, 1) for _ in range(input_size)] for _ in range(output_size)]
        self.biases = [[random.uniform(-1, 1)] for _ in range(output_size)]
        self.last_input = None
        self.last_z = None
        self.last_activation = None

    def dot_product(self, matrix, vector):
        result = []
        for row in matrix:
            value = sum(row[i] * vector[i][0] for i in range(len(row)))
            result.append([value])
        return result

    def add_vectors(self, a, b):
        return [[a[i][0] + b[i][0]] for i in range(len(a))]

    def apply_sigmoid(self, vector):
        return [[sigmoid(v[0])] for v in vector]

    def forward(self, input_data):
        self.last_input = input_data
        z = self.dot_product(self.weights, input_data)
        z = self.add_vectors(z, self.biases)
        self.last_z = z
        self.last_activation = self.apply_sigmoid(z)
        return self.last_activation

# ------------------ Neural Network Class ------------------ #
class NeuralNetwork:
    def __init__(self):
        self.layers = []

    def build_from_config(self, config_file_path):
        with open(config_file_path, 'r') as f:
            lines = f.readlines()
        num_layers = int(lines[0].strip())
        layer_sizes = [int(line.strip()) for line in lines[1:num_layers+1]]
        for i in range(1, len(layer_sizes)):
            self.layers.append(Layer(layer_sizes[i-1], layer_sizes[i]))

    def feedforward(self, input_data):
        activation = input_data
        for layer in self.layers:
            activation = layer.forward(activation)
        return activation

    def train(self, training_data, learning_rate=0.1, epochs=1000):
        for epoch in range(epochs):
            for x, y in training_data:
                # Forward pass
                activation = x
                for layer in self.layers:
                    activation = layer.forward(activation)

                # Backward pass
                # Output layer delta
                output = activation
                target = y
                delta = []
                for i in range(len(output)):
                    error = output[i][0] - target[i][0]
                    delta.append([error * sigmoid_derivative(self.layers[-1].last_z[i][0])])

                # Update weights and biases
                for l in reversed(range(len(self.layers))):
                    layer = self.layers[l]
                    input_to_use = layer.last_input
                    if l < len(self.layers) - 1:
                        next_layer = self.layers[l + 1]
                        new_delta = []
                        for i in range(len(layer.weights)):
                            error = sum(next_layer.weights[j][i] * delta[j][0] for j in range(len(next_layer.weights)))
                            new_delta.append([error * sigmoid_derivative(layer.last_z[i][0])])
                        delta = new_delta

                    # Gradient Descent Update
                    for i in range(len(layer.weights)):
                        for j in range(len(layer.weights[i])):
                            layer.weights[i][j] -= learning_rate * delta[i][0] * input_to_use[j][0]
                        layer.biases[i][0] -= learning_rate * delta[i][0]

    def predict(self, input_data):
        return self.feedforward(input_data)

# ------------------ Helper to Load CSV ------------------ #
def load_csv_data(filename):
    data = []
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            inputs = [[float(x)] for x in row[:-1]]
            output = [[float(row[-1])]]
            data.append((inputs, output))
    return data

# ------------------ Usage ------------------ #
nn = NeuralNetwork()
nn.build_from_config("network_config.txt")

# Load training data from CSV
training_data = load_csv_data("training_data.csv")

# Train the network
nn.train(training_data, learning_rate=0.5, epochs=10000)

# Test prediction
input_vector = [[0.5], [0.8]]
output = nn.predict(input_vector)
print("Prediction after training:\n", output)


FileNotFoundError: [Errno 2] No such file or directory: 'training_data.csv'